In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime as dt

In [2]:
base_url = 'https://www.transfermarkt.com/a/marktwertverlauf/spieler/' # "a" is a placeholder, can be anything, technically is player name but doesn't matter

In [3]:
headers = {'User-Agent': 
       'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [4]:
transfermarkt_ids = [148153, 82873, 209212]

In [5]:
columns = ['player_name', 'transfermarkt_player_id', 'date', 'market_value_in_gbp']

In [6]:
def get_market_value_history_of_player(transfermarkt_player_id, as_pandas=False):
    tree = requests.get(base_url + str(transfermarkt_player_id), headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    script = soup.find('script', text=re.compile('Highcharts.Chart')).text
    player_name = soup.find('title').text.partition(' - ')[0]
    
    chart_data = script.split("'data':[")[1]
    ending_string = '}}]'
    chart_data = chart_data[:chart_data.find(ending_string) + 1]

    delim = '},'
    market_values = [eval(e + delim[0]) for e in chart_data.split(delim) if e[:5] == "{'y':"]

    market_vals_clean = [(player_name, transfermarkt_player_id, dt.date.fromtimestamp(int(str(data_point['x'])[:-3])), data_point['y']) for data_point in market_values]

    if as_pandas:
        market_vals_clean = pd.DataFrame(market_vals_clean, columns=columns)

    return market_vals_clean

In [7]:
get_market_value_history_of_player(209212, as_pandas=True)

,player_name,transfermarkt_player_id,date,market_value_in_gbp
0,Solly March,209212,2014-03-07,75000
1,Solly March,209212,2014-08-19,150000
2,Solly March,209212,2015-02-24,150000
3,Solly March,209212,2015-09-24,200000
4,Solly March,209212,2016-04-20,200000
5,Solly March,209212,2016-09-01,200000
6,Solly March,209212,2017-02-27,250000
7,Solly March,209212,2017-06-28,500000
8,Solly March,209212,2017-10-23,2500000
9,Solly March,209212,2018-01-02,3500000


In [8]:
all_player_market_values = []

for id in transfermarkt_ids:
    all_player_market_values.extend(get_market_value_history_of_player(id))

In [9]:
df_all_player_market_values = pd.DataFrame(all_player_market_values, columns=columns)
df_all_player_market_values

,player_name,transfermarkt_player_id,date,market_value_in_gbp
0,Lewis Dunk,148153,2010-07-30,50000
1,Lewis Dunk,148153,2011-10-21,100000
2,Lewis Dunk,148153,2012-02-07,750000
3,Lewis Dunk,148153,2012-06-22,1000000
4,Lewis Dunk,148153,2013-03-02,1000000
...,...,...,...,...
73,Solly March,209212,2021-03-18,9000000
74,Solly March,209212,2021-06-08,9000000
75,Solly March,209212,2021-12-23,9000000
76,Solly March,209212,2022-06-15,9000000
